In [5]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
#mport seaborn as sns
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#import matplotlib.font_manager as fm
#font_path = 'fonts/MalgunGothic.ttf'
#font_name = fm.FontProperties(fname=font_path, size=10).get_name()
#plt.rc('font', family=font_name, size=12)
#plt.rcParams["figure.figsize"] = (20, 10)
#register_matplotlib_converters()

#mpl.font_manager._rebuild()
#mpl.pyplot.rc('font', family='NanumGothic')

In [7]:
val = pd.read_json('val.json', typ = 'frame', encoding='utf-8')
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [8]:
# playlist만 있는 것
idx = []
for i in range(len(val)):
    if val['plylst_title'].iloc[i]:
        if not val['tags'].iloc[i]:
            if not val['songs'].iloc[i]:
                idx.append(i)

In [9]:
idx

[1,
 9,
 35,
 57,
 71,
 101,
 115,
 135,
 167,
 199,
 218,
 220,
 221,
 240,
 253,
 254,
 257,
 279,
 300,
 304,
 307,
 308,
 317,
 333,
 334,
 352,
 379,
 384,
 391,
 400,
 416,
 420,
 430,
 440,
 454,
 466,
 474,
 475,
 481,
 490,
 498,
 540,
 547,
 572,
 598,
 601,
 613,
 615,
 633,
 648,
 675,
 680,
 681,
 683,
 693,
 704,
 732,
 736,
 759,
 770,
 779,
 796,
 805,
 807,
 819,
 861,
 869,
 877,
 908,
 909,
 912,
 942,
 951,
 956,
 978,
 985,
 1025,
 1028,
 1029,
 1037,
 1053,
 1058,
 1074,
 1079,
 1087,
 1090,
 1093,
 1116,
 1130,
 1165,
 1166,
 1193,
 1256,
 1262,
 1272,
 1281,
 1286,
 1287,
 1292,
 1306,
 1356,
 1374,
 1379,
 1425,
 1435,
 1444,
 1459,
 1476,
 1527,
 1529,
 1546,
 1549,
 1550,
 1562,
 1564,
 1570,
 1596,
 1597,
 1610,
 1626,
 1667,
 1671,
 1690,
 1729,
 1749,
 1750,
 1768,
 1774,
 1777,
 1792,
 1794,
 1807,
 1809,
 1820,
 1835,
 1848,
 1853,
 1855,
 1878,
 1894,
 1901,
 1905,
 1930,
 1950,
 1959,
 1963,
 1985,
 2003,
 2024,
 2061,
 2067,
 2090,
 2105,
 2131,
 2160

In [10]:
val.iloc[idx]

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
35,[],65114,"■■■■ 사랑,그리고이별 ■■■■",[],6,2010-10-27 10:34:34.000
57,[],87700,마쉬멜로우같은 멜로우한 음악,[],6,2016-01-14 10:19:30.000
71,[],35271,공부와 독서를 위한 #Newage,[],10,2020-01-17 15:46:20.000
...,...,...,...,...,...,...
22903,[],140513,10년이 지나 들어도 좋은 감성 Ballad,[],405,2016-01-11 10:58:05.000
22920,[],124704,가사의 의미와 뜻은모른다!! 오직 멜로디로만 선곡한 팝송!!,[],27,2016-02-05 12:31:59.000
22981,[],13045,＊카페 느낌 샹송♭,[],38,2011-07-12 00:58:39.000
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000


In [11]:
# playlist, tag만 있는 것
idx1 = []
for i in range(len(val)):
    if val['plylst_title'].iloc[i]:
        if val['tags'].iloc[i]:
            if not val['songs'].iloc[i]:
                idx1.append(i)
val.iloc[idx1]

,tags,id,plylst_title,songs,like_cnt,updt_date
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000
40,"[발렌타인데이, 달달한, 사랑노래, 고백]",80682,발렌타인 데이 -달콤한 초콜릿 매장 음악,[],27,2019-05-13 15:46:52.000
42,[새벽],8253,하루 끝 잔잔한 노래로 기분 좋게 잠에 들기,[],8,2019-06-08 23:48:41.000
...,...,...,...,...,...,...
22967,[분위기],36145,한국 갬성 있는 노래들만 모았다!!,[],12,2020-02-12 13:50:03.000
22969,"[감성, 신나는]",74670,노래 하나로 설레는 그런 발랄함,[],2,2020-03-11 00:55:41.000
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000


In [12]:
# songs, tag만 있는 것
idx2 = []
for i in range(len(val)):
    if not val['plylst_title'].iloc[i]:
        if val['tags'].iloc[i]:
            if val['songs'].iloc[i]:
                idx2.append(i)
val.iloc[idx2]

,tags,id,plylst_title,songs,like_cnt,updt_date
5,"[비, 분위기, 컨트리, 서정적인]",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
11,"[아침, 눈, 크리스마스]",136650,,"[174132, 257367, 539802, 568691, 296891, 38471...",6,2017-11-24 11:26:05.000
12,[EDM],53131,,"[392257, 668879, 460746, 680240, 490966, 25010...",268,2016-09-19 15:40:31.000
21,"[상큼한, 출근길, 트렌디, 팝, 신나는]",7373,,"[534729, 149593, 677181, 179617, 408079, 34339...",73,2019-12-17 15:47:59.000
23,[발렌타인데이],29647,,"[498353, 283397, 588606, 499171, 191963, 46258...",19,2017-02-12 22:25:12.000
...,...,...,...,...,...,...
23005,[휴가],136258,,"[678182, 405854, 342032, 637450, 680023, 32460...",5,2018-08-08 09:12:42.000
23007,"[휴식, 포크, 커피, 2000년대]",44013,,"[434841, 362936, 361453, 628693, 113135, 35071...",180,2019-12-17 17:55:11.000
23009,[알앤비],17766,,"[114387, 701557, 547449, 473514, 360916, 49845...",1,2015-07-21 17:06:45.000
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000


In [13]:
# songs만 있는 것
idx3 = []
for i in range(len(val)):
    if not val['plylst_title'].iloc[i]:
        if not val['tags'].iloc[i]:
            if val['songs'].iloc[i]:
                idx3.append(i)
val.iloc[idx3]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
6,[],127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000
...,...,...,...,...,...,...
23003,[],127349,,"[474446, 111476, 241237, 70500, 577313, 145058...",14,2018-02-02 17:43:37.000
23006,[],92067,,"[486938, 154124, 70969, 596414, 243733, 557848...",9,2013-02-16 01:07:41.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [21]:
9661+8975+2628+1745

23009

In [15]:
origin = val.index.tolist()

In [16]:
# 리스트에서 여러 값 삭제
def remove_values_from_list(the_list, lst):
    return [value for value in the_list if value not in lst]

origin1 = remove_values_from_list(origin, idx)
origin1

[0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,

In [17]:
origin2 = remove_values_from_list(origin1, idx1)

origin3 = remove_values_from_list(origin2, idx2)

origin4 = remove_values_from_list(origin3, idx3)
origin4

[4514, 6113, 12607, 17558, 19221, 22830]

In [18]:
val.iloc[origin4]

,tags,id,plylst_title,songs,like_cnt,updt_date
4514,[알앤비],120699,,[],3,2016-02-04 16:39:50.000
6113,[],108253,,[],28,2012-07-27 17:22:57.000
12607,[],2996,,[],229,2018-10-08 13:16:50.000
17558,[],125531,,[],2,2014-06-09 14:48:35.000
19221,"[봄, 연주곡, Morning, Instrumental, 아침]",15941,,[],13,2019-05-24 09:52:50.000
22830,[],22014,,[],3,2015-05-25 17:10:52.000


In [19]:
# playlist, song만 있는 것
idx100 = []
for i in range(len(val)):
    if val['plylst_title'].iloc[i]:
        if val['songs'].iloc[i]:
            if not val['tags'].iloc[i]:
                idx100.append(i)
val.iloc[idx100]

,tags,id,plylst_title,songs,like_cnt,updt_date


* association rule 리더보드에 제출하는 걸로 만들기(더 많은 feature 고려). 1,2번 케이스까지 어떻게 할 수 있을지 생각
* 발매일 연도별로 80년, 90년, 2000년대는 5년 단위로 2011년부터 1년 단위로 베스트셀러 노래 정리